In [1]:
#analytics vidya data supremacy hackathon
#imabalanced classificaton problem
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set()
%matplotlib inline

In [2]:
train_data = pd.read_csv('data/train_jqd04QH.csv')
test_data = pd.read_csv('data/test_GYi4Gz5.csv')
print "Training data ,",train_data.shape
print "Testing data ,",test_data.shape

Training data , (18359, 14)
Testing data , (15021, 13)


In [3]:
train_data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [4]:
###data preprocessing
Y = train_data.iloc[:,-1]
X = train_data.drop(['enrollee_id','city','target'],axis=1)
X.dtypes

city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
dtype: object

In [5]:
#missing values in each variable
X.apply(lambda x : sum(x.isnull()))

city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
dtype: int64

In [6]:
#helper functions
def random_impute_missing(X,variables):
    for var in variables:
        n_miss = sum(X[var].isnull())
        if n_miss == 0:
            continue
        uniq_values = np.array(X[var].value_counts().index)
        n_uniq = len(uniq_values)
        idx = np.random.randint(0,n_uniq,size = n_miss)
        values = uniq_values[idx]
        miss_idx = np.array(X[var][X[var].isnull()].index)
        X.loc[miss_idx,var] = values
    return X

def one_zero_converter(data,var,c):
    temp_x = list(set(data[var].unique()) - {np.nan,c})
    temp_idx = [x in temp_x for x in data[var]]
    data.loc[data[var] == c,var]=1.0
    data.loc[temp_idx,var]=0.0
    return data

In [7]:
X['major_discipline'].value_counts()

STEM               13738
Humanities           688
Other                343
Business Degree      307
Arts                 239
No Major             206
Name: major_discipline, dtype: int64

In [8]:
X = one_zero_converter(X,'major_discipline','STEM')
X['major_discipline'].value_counts()

1.0    13738
0.0     1783
Name: major_discipline, dtype: int64

In [9]:
X['company_size'].value_counts()

50-99        3120
100-500      2698
10000+       2044
10/49        1466
1000-4999    1399
<10          1360
500-999       902
5000-9999     591
Name: company_size, dtype: int64

In [10]:
X['company_type'].value_counts()

Pvt Ltd                10051
Funded Startup          1038
Public Sector            996
Early Stage Startup      582
NGO                      534
Other                    119
Name: company_type, dtype: int64

In [11]:
X = one_zero_converter(X,'company_type','Pvt Ltd')
X['company_type'].value_counts()

1.0    10051
0.0     3269
Name: company_type, dtype: int64

In [12]:
X['gender'].value_counts()

Male      12884
Female     1188
Other       189
Name: gender, dtype: int64

In [13]:
X = one_zero_converter(X,'gender','Male')
X['gender'].value_counts()

1.0    12884
0.0     1377
Name: gender, dtype: int64

In [14]:
missing_vars = ['gender','enrolled_university','education_level','major_discipline',
               'experience','company_size','company_type','last_new_job']

X = random_impute_missing(X,missing_vars)
X.apply(lambda x : sum(x.isnull()))

city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [15]:
X.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,0.689,1,Has relevent experience,no_enrollment,Graduate,1,3,100-500,1,1,106
1,0.923,1,Has relevent experience,no_enrollment,Graduate,1,14,<10,0,1,69
2,0.910,0,Has relevent experience,no_enrollment,Graduate,1,6,50-99,0,2,4
3,0.666,1,Has relevent experience,no_enrollment,Graduate,1,14,50-99,1,1,26
4,0.887,1,No relevent experience,no_enrollment,Masters,1,8,<10,1,2,88


In [16]:
###data preprocessing 
#convert data to numerical form
X['relevent_experience'].value_counts()

Has relevent experience    13596
No relevent experience      4763
Name: relevent_experience, dtype: int64

In [17]:
X = one_zero_converter(X,'relevent_experience','Has relevent experience')
X['relevent_experience'].value_counts()

1.0    13596
0.0     4763
Name: relevent_experience, dtype: int64

In [18]:
X['enrolled_university'].value_counts()

no_enrollment       13757
Full time course     3309
Part time course     1293
Name: enrolled_university, dtype: int64

In [19]:
X = one_zero_converter(X,'enrolled_university','Full time course')
X['enrolled_university'].value_counts()

0.0    15050
1.0     3309
Name: enrolled_university, dtype: int64

In [20]:
X['education_level'].value_counts()

Graduate          10861
Masters            4423
High School        2118
Phd                 559
Primary School      398
Name: education_level, dtype: int64

In [21]:
X.loc[X['education_level']=='Phd','education_level'] = 3.0
X.loc[X['education_level']=='Masters','education_level'] = 2.0
X.loc[X['education_level']=='Graduate','education_level'] = 1.0
X.loc[X['education_level']=='High School','education_level'] = 0.0
X.loc[X['education_level']=='Primary School','education_level'] = 0.0

In [22]:
X['education_level'].value_counts()

1.0    10861
2.0     4423
0.0     2516
3.0      559
Name: education_level, dtype: int64

In [23]:
X.loc[X['experience']=='>20','experience'] = 21.5
X.loc[X['experience']=='<1','experience'] = 0.5
X['experience'] = X['experience'].values.astype(np.float32)

In [24]:
X.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,0.689,1,1,0,1,1,3.0,100-500,1,1,106
1,0.923,1,1,0,1,1,14.0,<10,0,1,69
2,0.910,0,1,0,1,1,6.0,50-99,0,2,4
3,0.666,1,1,0,1,1,14.0,50-99,1,1,26
4,0.887,1,0,0,2,1,8.0,<10,1,2,88


In [25]:
X['company_size'].value_counts()

50-99        3705
100-500      3257
10000+       2632
10/49        2089
1000-4999    1991
<10          1978
500-999      1494
5000-9999    1213
Name: company_size, dtype: int64

In [26]:
X.loc[X['company_size']=='10000+','company_size'] = 7.0
X.loc[X['company_size']=='5000-9999','company_size'] = 5.0
X.loc[X['company_size']=='1000-4999','company_size'] = 3.5
X.loc[X['company_size']=='500-999','company_size'] = 2.5
X.loc[X['company_size']=='100-500','company_size'] = 1.75
X.loc[X['company_size']=='50-99','company_size'] = 1.0
X.loc[X['company_size']=='10/49','company_size'] = 0.5
X.loc[X['company_size']=='<10','company_size'] = 0.0

In [27]:
X['last_new_job'].value_counts()

1        7635
>4       3402
2        2885
never    2254
4        1099
3        1084
Name: last_new_job, dtype: int64

In [28]:
X.loc[X['last_new_job']=='>4','last_new_job'] = 5.0
X.loc[X['last_new_job']=='never','last_new_job'] = 0.0
X['last_new_job'] = X['last_new_job'].values.astype(np.float32)

In [29]:
X = X.astype(np.float32)

In [30]:
X.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,0.689,1.0,1.0,0.0,1.0,1.0,3.0,1.75,1.0,1.0,106.0
1,0.923,1.0,1.0,0.0,1.0,1.0,14.0,0.00,0.0,1.0,69.0
2,0.910,0.0,1.0,0.0,1.0,1.0,6.0,1.00,0.0,2.0,4.0
3,0.666,1.0,1.0,0.0,1.0,1.0,14.0,1.00,1.0,1.0,26.0
4,0.887,1.0,0.0,0.0,2.0,1.0,8.0,0.00,1.0,2.0,88.0


In [31]:
Y.head()

0    0
1    0
2    0
3    0
4    1
Name: target, dtype: int64

In [32]:
#conversions for test_set
test_data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,16548,city_33,0.448,NaN,No relevent experience,Full time course,Graduate,STEM,<1,1000-4999,Public Sector,NaN,15
1,12036,city_28,0.939,Male,No relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,94
2,11061,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,3,NaN,NaN,1,17
3,5032,city_104,0.924,Male,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,2,76
4,17599,city_77,0.830,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,<10,Pvt Ltd,2,65


In [33]:
test_X = test_data.drop(['enrollee_id','city'],axis=1)

In [34]:
test_X = one_zero_converter(test_X,'major_discipline','STEM')
test_X['major_discipline'].value_counts()

1.0    11117
0.0     1511
Name: major_discipline, dtype: int64

In [35]:
test_X = one_zero_converter(test_X,'gender','Male')
test_X['gender'].value_counts()

1.0    10578
0.0     1055
Name: gender, dtype: int64

In [36]:
test_X = one_zero_converter(test_X,'company_type','Pvt Ltd')
test_X['company_type'].value_counts()

1.0    8063
0.0    2628
Name: company_type, dtype: int64

In [37]:
missing_vars = ['gender','enrolled_university','education_level','major_discipline',
               'experience','company_size','company_type','last_new_job']

test_X = random_impute_missing(test_X,missing_vars)
test_X.apply(lambda x : sum(x.isnull()))

city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [38]:
test_X = one_zero_converter(test_X,'relevent_experience','Has relevent experience')
test_X['relevent_experience'].value_counts()

1.0    11102
0.0     3919
Name: relevent_experience, dtype: int64

In [39]:
test_X = one_zero_converter(test_X,'enrolled_university','Full time course')
test_X['enrolled_university'].value_counts()

0.0    12363
1.0     2658
Name: enrolled_university, dtype: int64

In [40]:
test_X.loc[test_X['education_level']=='Phd','education_level'] = 3.0
test_X.loc[test_X['education_level']=='Masters','education_level'] = 2.0
test_X.loc[test_X['education_level']=='Graduate','education_level'] = 1.0
test_X.loc[test_X['education_level']=='High School','education_level'] = 0.0
test_X.loc[test_X['education_level']=='Primary School','education_level'] = 0.0

In [41]:
test_X.loc[test_X['experience']=='>20','experience'] = 21.5
test_X.loc[test_X['experience']=='<1','experience'] = 0.5
test_X['experience'] = test_X['experience'].values.astype(np.float32)

In [42]:
test_X.loc[test_X['last_new_job']=='>4','last_new_job'] = 5.0
test_X.loc[test_X['last_new_job']=='never','last_new_job'] = 0.0
test_X['last_new_job'] = test_X['last_new_job'].values.astype(np.float32)

In [43]:
test_X.loc[test_X['company_size']=='10000+','company_size'] = 7.0
test_X.loc[test_X['company_size']=='5000-9999','company_size'] = 5.0
test_X.loc[test_X['company_size']=='1000-4999','company_size'] = 3.5
test_X.loc[test_X['company_size']=='500-999','company_size'] = 2.5
test_X.loc[test_X['company_size']=='100-500','company_size'] = 1.75
test_X.loc[test_X['company_size']=='50-99','company_size'] = 1.0
test_X.loc[test_X['company_size']=='10/49','company_size'] = 0.5
test_X.loc[test_X['company_size']=='<10','company_size'] = 0.0

In [44]:
test_X.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,0.448,0,0,1,1,1,0.5,3.5,0,4.0,15
1,0.939,1,0,0,1,1,21.5,1,1,1.0,94
2,0.920,1,0,1,1,1,3.0,1.75,1,1.0,17
3,0.924,1,0,0,3,1,21.5,1,1,2.0,76
4,0.830,1,1,0,1,1,6.0,0,1,2.0,65


In [45]:
test_X = test_X.astype(np.float32)

In [46]:
import pickle
f = open('dataset.pickle','w')
pickle.dump([X,test_X,Y],f)
f.close()